# PDF RAG - 소비자 분쟁조정 사례집 QA

## PDF Data

In [1]:
import os
from glob import glob
pdf_files = glob(os.path.join('pdf_data', '*분쟁*.pdf'))
len(pdf_files) 

3

In [2]:
# pdf 파일 목록
pdf_files

['pdf_data/2018 의료 분쟁조정 사례집.pdf',
 'pdf_data/2018 서비스·집단 분쟁조정 사례집.pdf',
 'pdf_data/2018 물품 분쟁조정 사례집.pdf']

In [3]:
# pdf 파일을 읽어서 텍스트로 변환
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf_files[1])
data = loader.load()
len(data)

200

In [4]:
# 첫번째 페이지의 텍스트 출력
from pprint import pprint
pprint(data[0].page_content) 

'소비자분쟁조정위원회2018\n서비스·집단 \n분쟁조정 사례집\n'


In [5]:
# 첫번째 페이지의 메타데이터 출력
pprint(data[0].metadata)  

{'page': 0, 'source': 'pdf_data/2018 서비스·집단 분쟁조정 사례집.pdf'}


In [6]:
# 10번째 페이지의 텍스트 출력
pprint(data[10].page_content)

('제1장\n'
 '일\n'
 '반분\n'
 '쟁\n'
 '조정\n'
 ' 사\n'
 '례 (\n'
 '서\n'
 '비스)\n'
 '제1장 일반분쟁조정 사례(서비스) ● 3\n'
 '사\n'
 '례01사건번호 2018일나565  | 결정일자 2018. 8. 7.\n'
 '세탁 후 갑피 마모 및 경화된 가죽 \n'
 '운동화에 대한 손해배상 요구\n'
 '주 문\n'
 '1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n'
 '운동화, 색상 : 흰색) 1켤레를 반환한다. \n'
 '2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n'
 '을 지급한다.\n'
 '이 유\n'
 '1. 기초사실\n'
 '가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n'
 '이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. 피신청인에게 이 사건 제품의 세탁을 '
 '의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n'
 '마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였\n'
 '으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, '
 '피신청인은 세탁과실이 없\n'
 '다는 이유로 이를 거부하였다.\n'
 '나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.\n'
 '   \n'
 '신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만\n'
 '으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제\n'
 '인지 단정하기 어려운바, 판단 불가하다.')


In [7]:
# 10번째 페이지의 텍스트에서 정보 추출
import re
split_pattern = r'사\n례\d+.*사건번호.*결정일자.*\d{4}\.\s?\d{1,2}\.\s?\d{1,2}\.'
split_text = re.findall(split_pattern, "".join(data[10].page_content))
split_text  

['사\n례01사건번호 2018일나565  | 결정일자 2018. 8. 7.']

In [8]:
# split_text가 존재하면 split_pattern으로 분리
if split_text:
    parts = re.split(split_pattern, "".join(data[10].page_content))

In [9]:
# 분리된 텍스트 출력
parts[0]

'제1장\n일\n반분\n쟁\n조정\n 사\n례 (\n서\n비스)\n제1장 일반분쟁조정 사례(서비스) ● 3\n'

In [10]:
# 분리된 텍스트 출력
parts[1]

'\n세탁 후 갑피 마모 및 경화된 가죽 \n운동화에 대한 손해배상 요구\n주 문\n1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n운동화, 색상 : 흰색) 1켤레를 반환한다. \n2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n을 지급한다.\n이 유\n1. 기초사실\n가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. 피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였\n으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, 피신청인은 세탁과실이 없\n다는 이유로 이를 거부하였다.\n나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.\n   \n신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만\n으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제\n인지 단정하기 어려운바, 판단 불가하다.'

In [11]:
# 특정 문자열의 위치 찾기
re.search(r'주 문\n', parts[1]).span()[0]

38

In [12]:
# 제목 추출
title = parts[1][:re.search(r'주 문\n', parts[1]).span()[0]].strip()
title

'세탁 후 갑피 마모 및 경화된 가죽 \n운동화에 대한 손해배상 요구'

In [13]:
# 내용 추출
content = parts[1][re.search(r'주 문\n', parts[1]).span()[0]:]
content

'주 문\n1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n운동화, 색상 : 흰색) 1켤레를 반환한다. \n2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n을 지급한다.\n이 유\n1. 기초사실\n가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. 피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였\n으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, 피신청인은 세탁과실이 없\n다는 이유로 이를 거부하였다.\n나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.\n   \n신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만\n으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제\n인지 단정하기 어려운바, 판단 불가하다.'

In [14]:
# 구분 문자열에서 사례 번호 추출
split_text[0]

'사\n례01사건번호 2018일나565  | 결정일자 2018. 8. 7.'

In [15]:
re.findall('례\s?(\d+)\s?사건번호', split_text[0])[0]

'01'

In [16]:
# 사건에 대한 메타데이터 추출하는 Pydantic 스키마 정의

from langchain_core.pydantic_v1 import BaseModel, Field

class Case(BaseModel):
    case_number: str = Field(description="The number of the case", required=True)
    case_date: str = Field(description="The date when the case was occurred (year, month, day)", required=True)   

In [17]:
# Extraction chain 구성
from langchain_core.prompts import PromptTemplate
from langchain_experimental.llms.ollama_functions import OllamaFunctions


prompt = PromptTemplate.from_template(
    """ Extract relevant information from the following text:
    
TEXT: {text} \n

AI: """
)

text = split_text[0]

llm = OllamaFunctions(model="llama3", format="json", temperature=0)
runnable = prompt | llm.with_structured_output(schema=Case)
response = runnable.invoke({"text": text})
print(response)

case_number='2018일나565' case_date='2018. 8. 7.'


In [18]:
dict(response)

{'case_number': '2018일나565', 'case_date': '2018. 8. 7.'}

In [19]:
data[-2].page_content   

'2018 서비스·집단 분쟁조정사례집\n인 쇄 / 2019년 5월\n발 행 / 2019년 5월발행인 / 한국소비자원 원장 이 희 숙\n편집인 / 소비자분쟁조정위원회 위원장 \n제 작 / 소비자분쟁조정위원회 분쟁조정사무국디자인 ․인쇄 / (사)아름다운사람들복지회\n발행처 / 한국소비자원 소비자분쟁조정위원회\n\U000f02b2\U000f02b7\U000f02b7\U000f02b3\U000f02b8\n충청북도 음성군 맹동면 용두로 54\n전 화 / 043-880-5500'

In [20]:
# 문서 객체를 페이지별로 순회하며 사례 번호 등 메타데이터를 추출하고 업데이트
pdf_docs = []
case_metadata = {}
for doc in data[10:-2]:
    split_text = re.findall(split_pattern, "".join(doc.page_content))
    if split_text:

        # case id
        case_metadata['case_id'] = re.findall('례\s?(\d+)\s?사건번호', split_text[0])[0]

        parts = re.split(split_pattern, "".join(doc.page_content))

        if re.search(r'주 문\n', parts[1]):
            # Add title to metadata
            case_metadata['title'] = parts[1][:re.search(r'주 문\n', parts[1]).span()[0]].replace('\n', '').strip()
                    
            # Update content
            doc.page_content = parts[1][re.search(r'주 문\n', parts[1]).span()[0]:].strip()
        else:
            case_metadata['title'] = ''
            
        # Extract metadata from text
        i = 0
        while i < 10:
            try:
                response = runnable.invoke({"text": split_text[0]})
                for k, v in dict(response).items():
                    case_metadata[k] = v.replace("\n", "").replace(" ", "")
                break
            except:
                i += 1
                continue

        # Update metadata 
        doc.metadata.update(case_metadata)

        # Append to split_docs
        pdf_docs.append(doc)

    else:
        # Update metadata 
        doc.metadata.update(case_metadata) 

        # Append to split_docs
        pdf_docs.append(doc)

len(pdf_docs)

188

In [21]:
pprint(pdf_docs[0].page_content)

('주 문\n'
 '1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n'
 '운동화, 색상 : 흰색) 1켤레를 반환한다. \n'
 '2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n'
 '을 지급한다.\n'
 '이 유\n'
 '1. 기초사실\n'
 '가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n'
 '이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. 피신청인에게 이 사건 제품의 세탁을 '
 '의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n'
 '마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였\n'
 '으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, '
 '피신청인은 세탁과실이 없\n'
 '다는 이유로 이를 거부하였다.\n'
 '나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.\n'
 '   \n'
 '신청인이 주장하는 갑피 벗겨짐(스크래치 등) 증상은 관찰되나 현 제품 상태만\n'
 '으로는 제품 훼손의 원인이 세탁 과정상 발생한 것인지 착화 환경에 따른 문제\n'
 '인지 단정하기 어려운바, 판단 불가하다.')


In [22]:
pprint(pdf_docs[0].metadata)

{'case_date': '2018.8.7.',
 'case_id': '01',
 'case_number': '2018일나565',
 'page': 10,
 'source': 'pdf_data/2018 서비스·집단 분쟁조정 사례집.pdf',
 'title': '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'}


## Embedding

In [24]:
# HugoingFace Embeddings를 다운로드
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
)

/Users/wiggler/Library/Caches/pypoetry/virtualenvs/langchain-v02-ycReq9Wb-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/wiggler/Library/Caches/pypoetry/virtualenvs/langchain-v02-ycReq9Wb-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
# 문서를 임베딩
embeddings = embeddings_model.embed_documents(
    [
        "온라인 쇼핑몰에서 주문한 제품이 불량품으로 배송되었습니다. 이에 대한 법적 책임은 누구에게 있나요?",
        "구입한 전자제품이 고장나서 환불을 요청했지만 거부당했습니다. 피해 보상을 받을 수 있나요?",
        "호텔 예약 후 도착했는데 예약이 취소되었다고 했습니다. 이에 대한 대응 방법은 무엇인가요?",
        "자동차 수리 후 동일한 문제가 재발했습니다. 수리업체에 대한 법적 조치를 취할 수 있나요?",
        "항공편이 지연되어 중요한 일정을 놓쳤습니다. 이에 대한 피해 보상을 받을 수 있나요?"
    ]
)
len(embeddings), len(embeddings[0])

(5, 768)

In [26]:
embedded_query = embeddings_model.embed_query("에어컨 제품 불량에 대해서 보상을 받을 수 있을까요?")
embedded_query[:5]

[0.7448063492774963,
 -0.2612632215023041,
 -0.25691506266593933,
 0.12548944354057312,
 -0.02586374618113041]

In [27]:
# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

# 쿼리와 문서 간의 코사인 유사도 계산
for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

0.45304341538135184
0.6305271996168532
0.3854962412100702
0.47294258162888303
0.3779668048938859


## Chunking

In [28]:
# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

text = "에어컨 제품 불량에 대해서 보상을 받을 수 있을까요?"
encoded = tokenizer.encode(text)
print(len(text), len(encoded))
print(encoded)

29 12
[2, 19224, 8918, 20035, 5051, 9665, 22898, 9622, 3317, 21981, 35, 3]


In [93]:
# Token 수를 기준으ㄹ 문서를 청크 단위로 분할
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 120,
    chunk_overlap  = 10,
)

split_docs = text_splitter.split_documents(pdf_docs)
print(len(split_docs))
print(split_docs[0])

905
page_content='주 문\n1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n운동화, 색상 : 흰색) 1켤레를 반환한다. \n2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n을 지급한다.\n이 유\n1. 기초사실' metadata={'source': 'pdf_data/2018 서비스·집단 분쟁조정 사례집.pdf', 'page': 10, 'case_id': '01', 'title': '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구', 'case_number': '2018일나565', 'case_date': '2018.8.7.'}


In [75]:
sample_text = split_docs[0].page_content
sample_encoded = tokenizer.encode(sample_text)
len(sample_text), len(sample_encoded)

(152, 71)

In [76]:
print(len(tokenizer.encode(split_docs[0].page_content)))
pprint(split_docs[0].page_content)

71
('주 문\n'
 '1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽 \n'
 '운동화, 색상 : 흰색) 1켤레를 반환한다. \n'
 '2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원\n'
 '을 지급한다.\n'
 '이 유\n'
 '1. 기초사실')


In [77]:
print(len(tokenizer.encode(split_docs[1].page_content)))
pprint(split_docs[1].page_content)

107
('1. 기초사실\n'
 '가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색, \n'
 '이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. 피신청인에게 이 사건 제품의 세탁을 '
 '의뢰(세탁비 4,000원)하였는데 수령 후 갑피 \n'
 '마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였')


In [78]:
# 마침표 뒤에 나오는 줄바꿈 문자는 그대로 두고 나머지 줄바꿈 문자만 제거
result = re.sub(r'(?<!\.)\n', ' ', split_docs[1].page_content)
result

'1. 기초사실 가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색,  이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. 피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피  마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였'

In [79]:
pprint(split_docs[0].metadata)

{'case_date': '2018.8.7.',
 'case_id': '01',
 'case_number': '2018일나565',
 'page': 10,
 'source': 'pdf_data/2018 서비스·집단 분쟁조정 사례집.pdf',
 'title': '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'}


In [80]:
pprint(split_docs[1].metadata)

{'case_date': '2018.8.7.',
 'case_id': '01',
 'case_number': '2018일나565',
 'page': 10,
 'source': 'pdf_data/2018 서비스·집단 분쟁조정 사례집.pdf',
 'title': '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'}


In [81]:
f"### 이 사건은 '{split_docs[1].metadata['title']}'에 대한 사례입니다."

"### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다."

# Indexing

In [82]:
final_docs = []
for doc in split_docs:
    doc.page_content = f"### 이 사건은 '{doc.metadata['title']}'에 대한 사례입니다.\n\n" + \
    re.sub(r'(?<!\.)\n', ' ', doc.page_content)
    final_docs.append(doc)

print(final_docs[0].page_content)

### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

주 문 1. 신청인은 2018. 10. 16.까지 피신청인에게 이 사건 제품(제품명 : ○○○○ 가죽  운동화, 색상 : 흰색) 1켤레를 반환한다.  2. 피신청인은 신청인으로부터 제1항 제품을 반환받음과 동시에 신청인에게 71,000원 을 지급한다.
이 유 1. 기초사실


In [83]:
print(final_docs[1].page_content)

### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

1. 기초사실 가. 신청인은 2017. 6. 6. 가죽 운동화(제품명 : ○○○○ 가죽 운동화, 색상 : 흰색,  이하 ‘이 사건 제품’) 1켤레를 160,200원에 구매하여 착화하였고, 2018. 1. 10. 피신청인에게 이 사건 제품의 세탁을 의뢰(세탁비 4,000원)하였는데 수령 후 갑피  마모 및 경화된 사실(이하 ‘이 사건 현상’)을 확인하여 피신청인이 재세탁을 하였


In [84]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=final_docs, 
                                    embedding=embeddings_model, 
                                    collection_name="consumer_case_qa",
                                    persist_directory="./chroma_db")

In [85]:
chroma_docs = vectorstore.similarity_search("세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?", k=5)
for doc in chroma_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:200])

01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

므로, 세탁과실에 따른 손해배상 및 세탁비 환급을 요구한다.
이에 대하여 피신청인은 이 사건 제품을 인수하였을 당시 이미 제품 상태가 좋지 않았 고, 한국소비자원의 신발제품심의위원회에서도 세탁과실로 인정하지 않았기 때문에 신 청인의 요구를 수용할 수 
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

세탁물에 대하여 세탁업자는 고객에게 세탁요금을 청구하지 못하므로, 세탁업자인 피신청인이 세탁비 4,000원을 신청인에게 환급이 상당하다.
이상을 종합하면, 신청인은 피신청인에게 이 사건 제품을 반환하고 피신청인은 손해배 상액 67,000원(112,14
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않아 피신청인에게 손해배상(세탁비 환급 포함)을 요구하였으며, 피신청인은 세탁과실이 없 다는 이유로 이를 거부하였다.
나. 한국소비자원 신발제품심의위원회 심의 결과는 다음과 같다.

01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

인지 단정하기 어려운바, 판단 불가하다.
02 15 ### 이 사건은 '해외 배송대행 중 파손된 제품에 대한 손해배상 요구'에 대한 사례입니다.

인정하지 아니함이 상당하다.
이상을 종합할 때, 피신청인은 2018. 10. 30.까지 피신청인의 비용으로 이 사건 제품 을 수거하고, 신청인에게 222,707원을 지급하며, 만일 피신청인이 위 지급을 지체하 면 2018. 10. 31.부터 다 갚는 날까지 「상법


## Retrieval

In [86]:
# Top K
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)

query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

므로, 세탁과실에 따른 손해배상 및 세탁비 환급을 요구한다.
이에 대하여
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

세탁물에 대하여 세탁업자는 고객에게 세탁요금을 청구하지 못하므로, 세탁업
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

인지 단정하기 어려운바, 판단 불가하다.
02 15 ### 이 사건은 '해외 배송대행 중 파손된 제품에 대한 손해배상 요구'에 대한 사례입니다.

인정하지 아니함이 상당하다.
이상을 종합할 때, 피신청인은 2018. 10. 30.까


In [87]:
# metadata를 이용한 필터링

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 5,
        'filter': {'case_id':'01'}
        }
)


query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

므로, 세탁과실에 따른 손해배상 및 세탁비 환급을 요구한다.
이에 대하여
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

세탁물에 대하여 세탁업자는 고객에게 세탁요금을 청구하지 못하므로, 세탁업
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

하여 그 손해가 발생 및 확대되었을 가능성을 배제할 수 없는 점 등에 비
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

살피건대, 피신청인은 세탁 전부터 이 사건 제품의 상태가 좋지 않았다고 


In [88]:
# page_content를 이용한 필터링

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 5,
        'where_document': {'$contains': '세탁'}
        }
)

query = "세탁 후 오염에 대한 손해배상은 어떻게 이루어지나요?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(str(doc.metadata["case_id"]), str(doc.metadata["page"]), doc.page_content[:100])

01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

므로, 세탁과실에 따른 손해배상 및 세탁비 환급을 요구한다.
이에 대하여
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

세탁물에 대하여 세탁업자는 고객에게 세탁요금을 청구하지 못하므로, 세탁업
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

하여 그 손해가 발생 및 확대되었을 가능성을 배제할 수 없는 점 등에 비
01 10 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

으나, 이후에도 경화현상만 다소 개선될 뿐 갑피 마모 현상이 개선되지 않
01 11 ### 이 사건은 '세탁 후 갑피 마모 및 경화된 가죽 운동화에 대한 손해배상 요구'에 대한 사례입니다.

살피건대, 피신청인은 세탁 전부터 이 사건 제품의 상태가 좋지 않았다고 


## Generation

In [89]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [90]:
# RAG Chain
llm = ChatOllama(model="qwen2", temperature=0)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

item = "세탁"
query = f"{item} 불량에 대한 손해배상은 어떻게 이루어지나요?"

retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': 2,
        'where_document': {'$contains': item}
        }
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


response = rag_chain.invoke(query)
response

'세탁 불량에 대한 손해배상은 다음과 같이 이루어집니다:\n\n1. 신청인은 세탁 후의 갑피 마모 및 경화된 가죽 운동화에 대해 손해배상을 요구합니다.\n2. 피신청인은 제품 상태가 좋지 않았고, 한국소비자원의 신발제품심의위원회에서도 세탁과실을 인정하지 않았다고 주장하며, 이로 인해 신청인의 요구를 수용할 수 없다고 주장합니다.\n3. 그러나 피신청인은 피해구제 담당자가 제시한 배상산정액 112,140원의 50%인 50,670원을 환급하기로 결정했습니다.\n\n이를 바탕으로 손해배상의 책임을 분석하면, 피신청인의 책임을 60%로 제한하는 것이 적절하다고 판단됩니다. 이는 공평하고 타당한 분담에 기반한 손해배상 제도와 원만한 해결을 위한 조정의 취지를 고려한 결정입니다.\n\n따라서, 세탁 불량에 대한 손해배상은 신청인과 피신청인 사이에서 책임 비율에 따라 이루어지며, 환급금액은 배상산정액의 50%로 결정되었습니다.'

In [91]:
print(response)

세탁 불량에 대한 손해배상은 다음과 같이 이루어집니다:

1. 신청인은 세탁 후의 갑피 마모 및 경화된 가죽 운동화에 대해 손해배상을 요구합니다.
2. 피신청인은 제품 상태가 좋지 않았고, 한국소비자원의 신발제품심의위원회에서도 세탁과실을 인정하지 않았다고 주장하며, 이로 인해 신청인의 요구를 수용할 수 없다고 주장합니다.
3. 그러나 피신청인은 피해구제 담당자가 제시한 배상산정액 112,140원의 50%인 50,670원을 환급하기로 결정했습니다.

이를 바탕으로 손해배상의 책임을 분석하면, 피신청인의 책임을 60%로 제한하는 것이 적절하다고 판단됩니다. 이는 공평하고 타당한 분담에 기반한 손해배상 제도와 원만한 해결을 위한 조정의 취지를 고려한 결정입니다.

따라서, 세탁 불량에 대한 손해배상은 신청인과 피신청인 사이에서 책임 비율에 따라 이루어지며, 환급금액은 배상산정액의 50%로 결정되었습니다.
